## Deep Research

One of the classic cross-business Agentic use cases! This is huge.

<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#00bfff;">Commercial implications</h2>
            <span style="color:#00bfff;">A Deep Research agent is broadly applicable to any business area, and to your own day-to-day activities. You can make use of this yourself!
            </span>
        </td>
    </tr>
</table>

In [16]:
# Imports

from agents import Agent, WebSearchTool, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel
from dotenv import load_dotenv
import asyncio
import os
from IPython.display import display, Markdown
from pprint import pprint
import requests
load_dotenv(override=True)

# Constants

pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

## OpenAI Hosted Tools

OpenAI Agents SDK includes the following hosted tools:

The `WebSearchTool` lets an agent search the web.  
The `FileSearchTool` allows retrieving information from your OpenAI Vector Stores.  
The `ComputerTool` allows automating computer use tasks like taking screenshots and clicking.

### Important note - API charge of WebSearchTool

This is costing me 2.5 cents per call for OpenAI WebSearchTool. That can add up to $2-$3 for the next 2 labs. We'll use low cost Search tools with other platforms, so feel free to skip running this if the cost is a concern.

Costs are here: https://platform.openai.com/docs/pricing#web-search

## We will be making 4 Agents:

1. Search Agent - searches online given a search term using an OpenAI hosted tool
2. Planner Agent - given a query from the user, come up with searches
3. Report Agent - make a report on results
4. Push Agent - send a notification to the user's phone with a summary

## Our First Agent: Search Agent

Given a Search term, search for it on the internet and summarize results.

In [2]:
INSTRUCTIONS = "You are a research assistant. Given a search term, you search the web for that term and \
produce a concise summary of the results. The summary must 2-3 paragraphs and less than 300 \
words. Capture the main points. Write succintly, no need to have complete sentences or good \
grammar. This will be consumed by someone synthesizing a report, so it's vital you capture the \
essence and ignore any fluff. Do not include any additional commentary other than the summary itself."

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)

In [3]:
message = "What are the most popular and successful AI Agent frameworks in Jul 2025"

with trace("Search"):
    result = await Runner.run(search_agent, message)

display(Markdown(result.final_output))

As of July 2025, several AI agent frameworks have gained prominence for their capabilities in developing sophisticated, multi-agent systems:

**LangChain**: A versatile framework that allows developers to build AI agents by chaining together language models, external tools, and custom logic. Its modular architecture supports complex workflows and integrations, making it suitable for applications like AI chat assistants and document Q&A bots. However, its resource-intensive nature and rapid evolution may require regular updates. ([appypieagents.ai](https://www.appypieagents.ai/blog/best-ai-agent-frameworks?utm_source=openai))

**AutoGen**: Developed by Microsoft, AutoGen facilitates the orchestration of multiple AI agents in collaborative, asynchronous environments. It enables role-based agents to communicate and share tasks, streamlining complex workflows. AutoGen is particularly effective for automated research, collaborative analytics, and AI workflow automation. Its integration with Azure and OpenAI services enhances its utility in enterprise settings. ([appypieagents.ai](https://www.appypieagents.ai/blog/best-ai-agent-frameworks?utm_source=openai))

**CrewAI**: This framework focuses on orchestrating role-based AI agents for collaborative tasks. CrewAI allows for the assignment of specific roles to agents, enabling them to work together efficiently on complex projects. It's particularly popular in customer service and marketing sectors, offering a straightforward approach to building multi-agent systems without complex dependencies. ([firecrawl.dev](https://www.firecrawl.dev/blog/best-open-source-agent-frameworks-2025?utm_source=openai))

**LangGraph**: An extension within the LangChain ecosystem, LangGraph specializes in building stateful, multi-participant applications by modeling steps as nodes and edges in a graph. It integrates seamlessly with LangChain, offering enhanced capabilities for complex workflows and long-term memory management. ([firecrawl.dev](https://www.firecrawl.dev/blog/best-open-source-agent-frameworks-2025?utm_source=openai))

**Google Agent Development Kit (ADK)**: Announced in April 2025, the Google ADK is a modular framework that integrates with Google's ecosystem, including Gemini and Vertex AI. It supports hierarchical agent compositions and custom tool development, facilitating the creation of sophisticated AI agents within Google's cloud infrastructure. ([firecrawl.dev](https://www.firecrawl.dev/blog/best-open-source-agent-frameworks-2025?utm_source=openai))

These frameworks represent the forefront of AI agent development, each offering unique features tailored to different application needs. 

Take a look at the trace

https://platform.openai.com/traces

## Our Second Agent: Planner Agent

Given a query, come up with 5 ideas for web searches that could be run.

Use Structured Outputs as our way to ensure the Agent provides what we need.

In [4]:
# See note above about cost of WebSearchTool

HOW_MANY_SEARCHES = 5

INSTRUCTIONS = f"You are a helpful research assistant. Given a query, come up with a set of web searches \
to perform to best answer the query. Output {HOW_MANY_SEARCHES} terms to query for."

# We use Pydantic objects to describe the Schema of the output

class WebSearchItem(BaseModel):
    reason: str
    "Your reasoning for why this search is important to the query."

    query: str
    "The search term to use for the web search."


class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem]
    """A list of web searches to perform to best answer the query."""

# We pass in the Pydantic object to ensure the output follows the schema

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)

In [ ]:

message = "What are the most popular and successful AI Agent frameworks in Aug 2025"

with trace("Search"):
    result = await Runner.run(planner_agent, message)
    pprint(result.final_output)

WebSearchPlan(searches=[WebSearchItem(reason='To identify the leading AI agent frameworks currently being used and their success metrics.', query='popular AI agent frameworks May 2025'), WebSearchItem(reason='To gather data on the most successful AI frameworks based on user adoption and performance.', query='successful AI frameworks statistics 2025'), WebSearchItem(reason='To find reviews and comparisons of various AI agent frameworks released or dominant in May 2025.', query='AI agent frameworks comparison May 2025'), WebSearchItem(reason='To explore developer preferences for AI frameworks and technologies in 2025.', query='top AI frameworks developer preferences 2025'), WebSearchItem(reason='To investigate the trends and innovations impacting AI agent frameworks in 2025.', query='AI agent frameworks trends innovations May 2025')])


## Our Third Agent: Writer Agent

Take the results of internet searches and make a report

In [ ]:
INSTRUCTIONS = (
    "You are a senior researcher tasked with writing a cohesive report for a research query. "
    "You will be provided with the original query, and some initial research done by a research assistant.\n"
    "You should first come up with an outline for the report that describes the structure and "
    "flow of the report. Then, generate the report and return that as your final output.\n"
    "The final output should be in markdown format, and it should be small and concise. Aim "
    "for 2-3 pages of content, not more than 1000 words."
)


class ReportData(BaseModel):
    short_summary: str
    """A short 2-3 sentence summary of the findings."""

    markdown_report: str
    """The final report"""

    follow_up_questions: list[str]
    """Suggested topics to research further"""


writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)

## Our Fourth Agent: push notification

Just to show how easy it is to make a tool!

I'm using a nifty product called PushOver - to set this up yourself, visit https://pushover.net

In [18]:
@function_tool
def push(message: str):
    """Send a push notification with this brief message"""
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)
    # Debugging: print payload and check for missing credentials
    if not pushover_user or not pushover_token:
        print("Pushover credentials are missing. Please set PUSHOVER_USER and PUSHOVER_TOKEN in your environment.")
        return {"status": "error", "reason": "missing credentials"}
    print("Sending push notification with payload:", payload)
    response = requests.post(pushover_url, data=payload)
    print("Pushover response:", response.status_code, response.text)
    if response.status_code != 200:
        return {"status": "error", "reason": response.text}
    return {"status": "success"}

In [10]:
push

FunctionTool(name='push', description='Send a push notification with this brief message', params_json_schema={'properties': {'message': {'title': 'Message', 'type': 'string'}}, 'required': ['message'], 'title': 'push_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x121c83b00>, strict_json_schema=True, is_enabled=True)

In [11]:
INSTRUCTIONS = """You are a member of a research team and will be provided with a short summary of a report.
When you receive the report summary, you send a push notification to the user using your tool, informing them that research is complete,
and including the report summary you receive"""


push_agent = Agent(
    name="Push agent",
    instructions=INSTRUCTIONS,
    tools=[push],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required")
)

### The next 3 functions will plan and execute the search, using planner_agent and search_agent

In [12]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for the query """
    print("Planning searches...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to run a web search for each item in the search plan """
    input = f"Search term: {item.query}\nReason for searching: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output

### The next 2 functions write a report and send a push notification

In [13]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to write a report based on the search results"""
    print("Thinking about report...")
    input = f"Original query: {query}\nSummarized search results: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing report")
    return result.final_output

async def send_push(report: ReportData):
    """ Use the push agent to send a notification to the user """
    print("Pushing...")
    result = await Runner.run(push_agent, report.short_summary)
    print("Push sent")
    return report

### Showtime!

In [15]:
# query ="What are the most popular and successful AI Agent frameworks in May 2025"
query = "Provide me detailed summary for iphone 17 and its variants"

with trace("Research trace"):
    print("Starting research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_push(report)  
    print("Hooray!")
display(Markdown(report.markdown_report))

Starting research...
Planning searches...
Will perform 5 searches
Searching...
Finished searching
Thinking about report...
Finished writing report
Pushing...
Push sent
Hooray!


# iPhone 17 Series Report

## Introduction
The iPhone 17 series, anticipated to launch in September 2025, is set to redefine user expectations with a combination of aesthetic designs and high-performance technologies. Featuring four distinct models—the iPhone 17, iPhone 17 Air, iPhone 17 Pro, and iPhone 17 Pro Max—Apple aims to cater to a diverse range of consumer preferences and needs. This report offers a detailed summary of each model, discussing specifications, key features, pricing, and user feedback expected upon release.

## 1. Overview of the iPhone 17 Series
The iPhone 17 series marks Apple's latest innovation in smartphone technology, characterized by enhanced performance, improved camera systems, and a sophisticated design mentality. As rumors suggest, all four models will share consistent design aesthetics while offering distinctive features tailored to different segments of the smartphone market.

### 1.1 Launch Timeline
- **Expected Announcement:** September 2025
- **Pre-orders Start:** September 12, 2025
- **Official Release Date:** September 19, 2025

## 2. Detailed Model Specifications
### 2.1 iPhone 17
- **Display:** 6.3-inch OLED with ProMotion technology supporting up to 120Hz refresh rate.
- **Camera:** 24MP front camera and improved rear camera functionality.
- **Processor:** Powered by the A19 chip.
- **Price Range:** Starting at $849.
- **Key Features:** Enhanced haptic feedback and improved durability.

### 2.2 iPhone 17 Air
- **Display:** 6.6-inch LTPO OLED display.
- **Camera:** Equipped with a single 48MP rear camera.
- **Thickness:** Approximately 5.5mm, positioning it as Apple’s thinnest smartphone to date.
- **Battery:** A 2,800mAh battery optimized for improved endurance.
- **Price Range:** Estimated between $899 to $1,299.
- **Critiques:** Some concerns regarding battery life and durability due to its slim design.

### 2.3 iPhone 17 Pro
- **Display:** 6.3-inch screen with ProMotion technology, similar to the standard model.
- **Processor:** Featuring the A19 Pro chip.
- **Camera System:** Triple 48MP rear camera configuration.
- **Key Features:** 12GB of RAM with computational photography enhancements.
- **Price Range:** Starting at $999.

### 2.4 iPhone 17 Pro Max
- **Display:** 6.9-inch Retina display with enhanced Dynamic Island.
- **Camera:** A triple 48MP camera system with a telephoto lens supporting 8x optical zoom.
- **Processor:** Enhanced performance with A19 Pro chip.
- **Price Range:** Expected to start at $1,199.
- **Innovations:** Introduction of a vapor chamber cooling system to enhance thermal performance.

## 3. Key Features Across All Models
- **Operating System:** All models will ship with iOS 26, introducing advanced AI capabilities.
- **Charging Capabilities:** Support for Qi2 wireless charging up to 35W across all models.
- **Storage Options:** Varying base storage capabilities, with Pro models expected to have higher capacities.
- **Durability:** Enhanced water and dust resistance specifications.
- **Color Choices:** New color options including Steel Gray, Green, and variations for the Pro models such as Dark Blue and Orange.

## 4. Market Expectations and User Feedback
As the launch approaches, user anticipation is mixed with skepticism regarding certain models, especially the iPhone 17 Air, which has received criticism for potential battery and performance issues. Early insights note:
- **Positive Feedback:** Users appreciate the ergonomics of the iPhone 17 standard model for its comfortable fit and aesthetic.
- **Concerns for iPhone 17 Air:** Early reports highlight freezing and app malfunctions, raising concerns about reliability and efficiency.

## 5. Comparative Analysis
A comparative review of the iPhone 17 models showcases strengths and weaknesses:
- **Standard Model vs. Air:** The iPhone 17 offers a more robust camera system and better build quality compared to the ultrathin Air variant.
- **Pro vs. Pro Max:** The primary distinction lies in the screen size and advanced camera features, making the Pro Max the go-to option for professional users or those prioritizing photography.

## 6. Conclusion
The iPhone 17 series promises to bring significant innovations and variety to consumers. Each model is equipped with unique features that cater to an array of user preferences—from the sleek iPhone 17 Air to the advanced capabilities of the iPhone 17 Pro Max. As final specifications and user experiences unfold, consumers should weigh their needs against the features offered by each model to make an informed purchasing decision. Apple continues to set the benchmark for smartphone technology, and the iPhone 17 series is poised to uphold this legacy.

## 7. References
- [Tom's Hardware](https://www.tomsguide.com/phones/iphones/iphone-17-rumors-everything-we-know-so-far?utm_source=openai)
- [Cincodias](https://cincodias.elpais.com/smartlife/smartphones/2025-08-11/imagen-dimensiones-pantalla-iphone-17.html?utm_source=openai)
- [Economic Times](https://www.economictimes.com/news/international/us/apple-may-introduce-an-ultra-thin-iphone-air-to-replace-the-plus-model-/amp_articleshow/123244554.cms?utm_source=openai)

---
This detailed report aims to prepare prospective buyers for the iPhone 17 series, aiding them in navigating the multitude of options and enhancing their understanding of the cutting-edge technology Apple continues to innovate.

### As always, take a look at the trace

https://platform.openai.com/traces